# Orders & GMV

In [1]:
# ## for jupyter notebook only
# import warnings
# warnings.simplefilter('ignore') ## Importing libraries

# from IPython.display import display, HTML
# display(HTML(data="""
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 95%; }
#     div#maintoolbar-container { width: 95%; }
# </style>
# """))


# Importing libraries
import pandas as pd
import numpy as np
import os
import pygsheets
from ds.utilities.io import ds_trino as ds_presto
from os import listdir
from os.path import isfile, join
from google.cloud import storage
from datetime import timedelta, date, datetime 
import json
import requests
import urllib

# pd.options.display.max_columns = None
# pd.options.display.max_rows = 50
# pd.options.mode.chained_assignment = None

conn = ds_presto.create_trino_connection()

g_sheets_creds_path = '/home/guilherme.augusto/creds/gsheets-google-service-account.json'

wiw_creds_path = "/home/guilherme.augusto/creds/wiw_creds.json"

In [2]:
## BRAZIL/COLOMBIA LIGHTTECH/RAPPI CLEANING FUNCTIONS

def facility(x):
    try:
        if x.strip() == 'BR - Sao Paulo - Clélia':
            return 'SAO - Lapa Clelia'
        elif x.strip() == 'BR - Sao Paulo - Acre':
            return 'SAO - Rua do Acre'
        elif x.strip() == 'SA-BR-RJ-José Bonifacio':
            return 'RIO - Jose Bonifacio'
        elif x.strip() == 'BR - Sao Paulo - Guararapes':
            return 'SAO - Guararapes (Brooklin)'
        elif x.strip() == 'BR - Sao Paulo - Dino':
            return 'SAO - Dino'
        elif x.strip() == 'SA-BR-BH-Av General David Sarnoff':
            return 'BHZ - Contagem Industrial'
        elif x.strip() == 'SA-BR-BH-Av-Artur-Guimarães':
            return 'BHZ - Arthur Guimaraes'
        elif x.strip() == 'SA-BR-BH-Rua Paracatu':
            return 'BHZ - Paracatu'
        elif x.strip() == 'BR - Sao Paulo - Borba':
            return 'SAO - Borba'
        elif x.strip() == 'SA-BR-RJ-Uptown':
            return 'RIO - Barra'
        elif x.strip() == 'SA-BR-RJ-Camerino':
            return 'RIO - Camerino'
        elif x.strip() == 'Tancredo Neves':
            return 'BHZ - Tancredo Neves'
        elif x.strip() == 'SA-CO-Bogotá-Castellana':
            return 'BOG - Castellana'
        elif x.strip() == 'SA-CO-Cali-Cali Norte':
            return 'CLO - Chipichape'
        elif x.strip() == 'SA-CO-BAQ-Alto Prado':
            return 'BAQ - Alto Prado'
        elif x.strip() == 'SA-CO-Bogotá-Atabanza':
            return 'BOG - Atabanza'
        elif x.strip() == 'SA-CO-Bogota-San Felipe':
            return 'BOG - San Felipe'
        elif x.strip() == 'SA-CO-Bogotá-Puente Aranda':
            return 'BOG - Puente Aranda'
        elif x.strip() == 'SA-BR-RJ-Niteroi':
            return 'RIO - Niteroi'
        elif x.strip() == 'SA-BR-RJ-Tijuca':
            return 'RIO - Tijuca' 
        elif x.strip() == 'SA-BR-SP-Guarulhos':
            return 'SAO - Espanhola'        
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan
      

    
def city(x):
    try:
        belo = ['BHZ - Contagem Industrial', 'BHZ - Paracatu', 'BHZ - Arthur Guimaraes', 'BHZ - Tancredo Neves']
        sao = ['SAO - Rua do Acre', 'SAO - Dino', 'SAO - Guararapes (Brooklin)', 'SAO - Lapa Clelia', 'SAO - Borba', 'SAO - Vargas','SAO - Espanhola']
        rio = ['RIO - Jose Bonifacio', 'RIO - Barra', 'RIO - Camerino', 'RIO - Niteroi','RIO - Tijuca' ]
        bog = ['BOG - Prado', 'BOG - Castellana', 'BOG - Cocinas de la 73', 'BOG - Puente Aranda', 'BOG - Atabanza', 'BOG - San Felipe']
        cal = ['CLO - Chipichape']
        bar = ['BAQ - Alto Prado']
        med = ['MED - DModa']
        if x.strip() in belo:
            return 'Belo Horizonte'
        elif x.strip() in sao:
            return 'Sao Paulo'
        elif x.strip() in rio:
            return 'Rio de Janeiro'
        elif x.strip() in bog:
            return 'Bogota'
        elif x.strip() in cal:
            return 'Cali'
        elif x.strip() in med:
            return 'Medellin'
        elif x.strip() in bar:
            return 'Barranquilla'
        else:
            return np.nan
    except:
        return np.nan
    
    

In [3]:
query = """
WITH first_order as (
SELECT 
store_id, 
MIN(reference_time/1000) as first_order
FROM hudi_ingest.analytics_views.customer_orders co
WHERE day_partition >= '2020-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2020-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Espanhola
             
            
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1
            )
            
, last_order as (
SELECT 
store_id, 
MAX(reference_time/1000) as last_order
FROM hudi_ingest.analytics_views.customer_orders co
WHERE day_partition >= '2020-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2020-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1
            )
            
, aov_fix as (

SELECT 
dc.country as country,
store_id,
AVG(subtotal) aov_fix

FROM hudi_ingest.analytics_views.customer_orders co
LEFT JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
    
WHERE day_partition >= '2020-01-01'
AND (subtotal is not null or subtotal != 0)
AND NOT is_cancelled
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2020-01-01'
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
        -- MEDELLIN
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1, 2

)

SELECT 
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_week,
'LATAM' as region,
dc.country,
(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849',   -- LIM - Magdalena
         '4e36823e-98a2-4382-8797-48ef313196a9' -- LIM - Lince
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82'  -- BOG - Cedritos
      )
   THEN
      'Bogota'
   WHEN 
      facility_id in 
      (
         '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' -- MED - DModa
      )
   THEN
       'Medellin'
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9',  -- SJO - Pavas
         '035c0131-50e4-465a-9596-ad8eb9be03f8'   -- SJO - Heredia
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee'   -- GDL - Av. Mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         '89cf86b9-9e74-4657-86df-2a851492af88' -- SAO - Guarulhos
      )
   then
      'Sao Paulo'
    WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   WHEN facility_id = '4e36823e-98a2-4382-8797-48ef313196a9' THEN 'LIM - Lince'
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   WHEN facility_id = '035c0131-50e4-465a-9596-ad8eb9be03f8' THEN 'SJO - Heredia'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
   
   

   
   
   
END
) AS facility_name,
organization_name,
organization_id,
tenant_type,
-- date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) < date_add('week', 4, date_trunc('week', DATE(from_unixtime(first_order, facility_timezone)))) as is_new_store,
COUNT (DISTINCT order_id) AS all_orders,
COUNT (DISTINCT CASE WHEN is_cancelled THEN order_id END) AS cancelled_orders,
COUNT (DISTINCT order_id)-COUNT (DISTINCT CASE WHEN is_cancelled THEN order_id END) AS orders,
SUM(coalesce(subtotal, ao.aov_fix)) AS all_gmv,
SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, ao.aov_fix) ELSE 0 END) AS cancelled_gmv,
SUM(coalesce(subtotal, ao.aov_fix)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, ao.aov_fix) ELSE 0 END) AS gmv,
(SUM(coalesce(subtotal, ao.aov_fix)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, ao.aov_fix) ELSE 0 END)) / (COUNT (DISTINCT order_id)-COUNT (DISTINCT case when is_cancelled  THEN order_id END)) AS aov,
SUM(usd_subtotal ) AS all_gmv_usd,
SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END) AS cancelled_gmv_usd,
SUM(usd_subtotal )-SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END) AS gmv_usd,
(SUM(usd_subtotal )-SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END)) / (COUNT (DISTINCT order_id)-COUNT (DISTINCT CASE WHEN is_cancelled = TRUE THEN order_id END)) AS aov_usd,
SUM(CASE WHEN NOT is_cancelled THEN coalesce(net_payout, ao.aov_fix) ELSE 0 END) AS net_payout,
SUM(CASE WHEN NOT is_cancelled THEN abs(discount) ELSE 0 END) AS discount,
(SUM(coalesce(subtotal, ao.aov_fix)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, ao.aov_fix) ELSE 0 END)) - SUM(CASE WHEN NOT is_cancelled THEN abs(discount) ELSE 0 END) AS net_gmv

FROM hudi_ingest.analytics_views.customer_orders co
LEFT JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
LEFT JOIN first_order fo 
   ON co.store_id=fo.store_id
LEFT JOIN last_order lo 
   ON co.store_id=lo.store_id
LEFT JOIN aov_fix ao 
   ON co.store_id=ao.store_id AND dc.country = ao.country
    
WHERE day_partition >= '2020-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2020-01-01'
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
        -- MEDELLIN
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- HEREDIA 
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1,2,3,4,5,6,7,8,9
ORDER BY 1 DESC,2 ,4,5,6,7
"""

print('Querying Latam Orders & GMV...')
latam = ds_presto.fetch_data(conn=conn, sql_str=query, use_cache=False)
print('Latam Orders & GMV Pulled ...')


Querying Latam Orders & GMV...
Latam Orders & GMV Pulled ...


In [4]:
# converting column numbers to numeric type
latam.aov = pd.to_numeric(latam.aov ,errors='coerce')


# If tenant type is null, return DELIVERY. 
latam.tenant_type = np.where(latam.tenant_type.isna(), 'DELIVERY', latam.tenant_type)

# If brand is null, return FALSE to the is new brand column. 
# latam.is_new_store = np.where(latam.is_new_store.isna(), False, latam.is_new_store)


In [5]:
## Getting all lighttech information from BRA

## GSheets Oauth
gc = pygsheets.authorize(service_file=g_sheets_creds_path)

############# SÃO PAULO ############################################

# ACRE
acre = gc.open_by_key('1u8N3EHnRxpRB3o12kkk89qCXMLYV_erAHJS9nvyx_jM')
wsacre = acre.worksheet('title','Feed Runner')
acredf = wsacre.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# CLELIA
clelia = gc.open_by_key('1Co1FEKw_TOTE1FFp_WBBvQg3wQt3w256_rD0nCGjbJY')
wsclelia = clelia.worksheet('title','Feed Runner')
cleliadf = wsclelia.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# DINO
dino = gc.open_by_key('1x9rbysjVVL0Y8LDEkQioWgG-CmpEfovWMsBfD33CZXU')
wsdino = dino.worksheet('title','Feed Runner')
dinodf = wsdino.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# GUARARAPES
guarara = gc.open_by_key('1QT4hdgZe4lwE19eo8yQ6LS6Rv2ozvOycjAy6DkjBTn4')
wsguarara = guarara.worksheet('title','Feed Runner')
guararadf = wsguarara.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# BORBA
borba = gc.open_by_key('1an2a2bnzJ_iqenuMiCQvEXdUcJMz8KwaEL21OriZYW8')
wsborba = borba.worksheet('title','Feed Runner')
borbadf = wsborba.get_as_df(has_header=True,start="B8", end=None,empty_value='')


############# BELO HORIZONTE ############################################

# INDUSTRIAL
industrial = gc.open_by_key('1My9Ph_c0nM7eYASIIjcYy1cFw8Ip_T4rhOUgnM73O0M')
wsindustrial = industrial.worksheet('title','Repor')
industrialdf = wsindustrial.get_as_df(has_header=True,start="A1", end=None,empty_value='')

# ARTHUR GUIMARÃES
ag = gc.open_by_key('1wC3GcubNMLIa4pDT0HxRfdfucQZo5PCHXzifwLZHNgE')
wsag = ag.worksheet('title','Feed Runner')
agdf = wsag.get_as_df(has_header=True,start="B8", end=None,empty_value='')


# PARACATU
paracatu = gc.open_by_key('1EVuoGK-EAixIFXYu8jLdjEgZpIdkzR_5AeB55PLMsME')
wsparacatu = paracatu.worksheet('title','Feed Runner')
paracatudf = wsparacatu.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# CASTELO
castelo = gc.open_by_key('1gkbjx3nHJuDNd1QaJu0GwrSz4oe6qsKulnFIQmc1PSo')
wscastelo = castelo.worksheet('title','Feed Runner')
castelodf = wscastelo.get_as_df(has_header=True,start="B8", end=None,empty_value='')


############# RIO DE JANEIRO ############################################


# MEIER
meier = gc.open_by_key('1deVBnnKTjU5zhxj2S3PGPaTmbrPfHmvF08U6_8BZm9Y')
wsmeier = meier.worksheet('title','Feed Runner')
meierdf = wsmeier.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# BARRA
barra = gc.open_by_key('10ODauBi60mpfxiqIQ76vSxR2JJNmBnkxzCSzYGHufWI')
wsbarra = barra.worksheet('title','Feed Runner')
barradf = wsbarra.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# CAMERINO
camerino = gc.open_by_key('1KP6d_NKPMfYafW1b_i62O1PIx6n0zJUW-mSUWf468XU')
wscamerino = camerino.worksheet('title','Feed Runner')
camerinodf = wscamerino.get_as_df(has_header=True,start="B8", end=None,empty_value='')


# NITEROI
niteroi = gc.open_by_key('1jOlzMt4a6rBYcgIpwel5uZTFOE6XZBriTJnDxsLBi1g')
wsniteroi = niteroi.worksheet('title','Feed Runner')
niteroidf = wsniteroi.get_as_df(has_header=True,start="B8", end=None,empty_value='')



# GUARULHOS
guaru_hist = gc.open_by_key('1Rbl7xZazZgmsF9nW4bWnjXrz86zr94nbCkOftIl2bMw')
wsguaru_hist = guaru_hist.worksheet('title','Feed Runner')
guarudf_hist = wsguaru_hist.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# TIJUCA
tijuca_hist = gc.open_by_key('1xIlU8LWuhYMI4X9yDvT7K060QU89RbSSkLnIQ5wT2oY')
wstijuca_hist = tijuca_hist.worksheet('title','Feed Runner')
tijucadf_hist = wstijuca_hist.get_as_df(has_header=True,start="B8", end=None,empty_value='')








# dataframes to list
dataframes = [acredf, cleliadf, dinodf, guararadf, industrialdf, agdf, 
              paracatudf, meierdf, barradf, borbadf, camerinodf, castelodf,
              niteroidf,guarudf_hist,tijucadf_hist]


/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pypi__pygsheets/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pypi__pygsheets/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing

In [6]:
niteroidf.head()

,Número \ndo pedido,Restaurante,Organization,Aplicativo,Valor do pedido\n(use vírgula),Facility,Hora de Registro,Número pedido,
0,NITEROI9,American Burger - KC,American Burger - KC,UberEats,"R$ 37,97",SA-BR-RJ-Niteroi,2021-10-27 22:18:40,077F7,
1,NITEROI10,American Burger - KC,American Burger - KC,UberEats,"R$ 7,99",SA-BR-RJ-Niteroi,2021-10-27 22:20:05,18DC6,
2,NITEROI11,American Burger - KC,American Burger - KC,UberEats,"R$ 8,99",SA-BR-RJ-Niteroi,2021-10-27 22:21:49,08EB1,
3,NITEROI12,American Burger - KC,American Burger - KC,UberEats,"R$ 7,99",SA-BR-RJ-Niteroi,2021-10-27 22:49:32,E5FOA,
4,NITEROI13,American Burger - KC,American Burger - KC,UberEats,"R$ 7,99",SA-BR-RJ-Niteroi,2021-10-27 23:37:55,74B18,


In [7]:
# concatenating lighttech DataFrames
lighttech = pd.concat(dataframes)


# renaming columns to match Presto Query (latam df) headers
lighttech.rename(columns={'Número \ndo pedido':'orders', 
                         'Organization':'organization_name',
                         'Valor do pedido\n(use vírgula)':'gmv',
                         'Facility':'facility_name',
                         'Hora de Registro':'created_date'}, inplace=True)


# eliminating unused columns
lighttech = lighttech[['created_date', 'orders', 'facility_name', 'organization_name', 'gmv']]

# replacing whitespaces with NaN
lighttech = lighttech.replace(r'^\s*$', np.nan, regex=True)

# cleaning dates
lighttech['created_date'] = lighttech['created_date'].str.replace('/', '-').str.replace('--', '-')

# converting to datetime
lighttech['created_date'] = pd.to_datetime(lighttech['created_date'], errors = 'coerce')

# eliminating rows without date or facility name
lighttech.dropna(subset=['created_date', 'facility_name'], inplace=True)

# cleaning the order amout column. Removing the R$ and replacing comma with dot. 
lighttech['gmv'] = lighttech['gmv'].astype(str).apply(lambda x: x.replace("R$","").replace(",", "."))

# converting to numeric type the gmv column. All errors will be set as NAN
lighttech['gmv'] = pd.to_numeric(lighttech['gmv'] ,errors='coerce')


# removing the time part from the timestamp
lighttech.created_date = lighttech.created_date.dt.date
lighttech.created_date = lighttech.created_date.astype('datetime64[ns]')


# getting first day of week
lighttech['dayofweek'] = lighttech.created_date.dt.weekday
lighttech['created_week'] = (lighttech.created_date - lighttech.dayofweek * timedelta(days=1))

# getting first day of month
lighttech['created_month'] = lighttech.created_date.dt.to_period('M').dt.to_timestamp()


<ipython-input-7-6bf188001fe8>:23: UserWarning: Parsing '16-09-2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  lighttech['created_date'] = pd.to_datetime(lighttech['created_date'], errors = 'coerce')
<ipython-input-7-6bf188001fe8>:23: UserWarning: Parsing '26-02-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  lighttech['created_date'] = pd.to_datetime(lighttech['created_date'], errors = 'coerce')


In [8]:
# getting exchange rate
path_cred = '/home/guilherme.augusto/Credencials/client_secret.json'
gc = pygsheets.authorize(service_file= path_cred)
dl = gc.open_by_key('1izx9CVQrxsmxpBwgfTdIPVQx7EB2oZkjAEozioP-PGA')
dlx = dl.worksheet('title','USD')
dldf = dlx.get_as_df(has_header=True,start="A1", end=None,empty_value='')
dldf.rename(columns={'Close':'USD'}, inplace=True)

In [9]:
dldf.head()

,Date,USD
0,2020-05-01,5.4858
1,2020-05-02,5.4859
2,2020-05-03,5.4770
3,2020-05-04,5.5406
4,2020-05-05,5.5781


In [10]:


# preparing the merge and merging
dldf['Date'] = pd.to_datetime(dldf['Date'])

lighttech = pd.merge(lighttech, dldf, left_on="created_date", right_on="Date", how="left")

lighttech['gmv_usd'] = lighttech.gmv / lighttech.USD
# lighttech['aov_usd'] = lighttech.gmv_usd / lighttech.orders
# lighttech.orders = pd.to_numeric(lighttech.orders)


In [11]:
# grouping to get amount of orders and gmv
lighttech = lighttech.groupby(['created_month', 'created_week',
                      'facility_name',
                      'organization_name']
                     , as_index=False
                     , dropna=False
                     ).agg({'orders': 'count',
                            'gmv': 'sum', 
                           'gmv_usd': 'sum'}).reset_index(drop=True)

lighttech.orders = pd.to_numeric(lighttech.orders)

# creating new columns to match the Presto query headers
lighttech['region'] = 'LATAM'
lighttech['country'] = 'Brazil'
lighttech['tenant_type'] = 'DELIVERY'
# lighttech['is_new_store'] = False
lighttech['all_orders'] = lighttech.orders
lighttech['cancelled_orders'] = 0
lighttech['all_gmv'] = lighttech.gmv
lighttech['cancelled_gmv'] = 0
lighttech['aov'] = lighttech.gmv / lighttech.orders
lighttech['all_gmv_usd'] = np.nan
lighttech['cancelled_gmv_usd'] = np.nan
lighttech['aov_usd'] = lighttech.gmv_usd / lighttech.orders


In [12]:
lighttech['facility_name'] = lighttech['facility_name'].apply(facility)
lighttech['city'] = lighttech['facility_name'].apply(city)
lighttech['organization_id'] = np.nan
lighttech['net_payout'] = lighttech.gmv
lighttech['net_gmv'] = lighttech.gmv
lighttech['discount'] = 0


# ordering the columns to match the Presto query result
lighttech = lighttech[[c for c in latam.columns]]

In [13]:
############# RAPPI

# RAPPI
rappi = gc.open_by_key('1XfvvARTR0-fNvgcy8UnXxfTWIxFifCF5XQpJW5lRctE')
wsrappi = rappi.worksheet('title','Data - Rappi Shopper Monthly')
rappidf = wsrappi.get_as_df(has_header=True,start="A", end="AD",empty_value='')

rappidf.rename(columns={'month':'created_month', 
                         'week':'created_week',
                         'organization':'organization_name',
                         # 'Facility':'facility_name',
                         'PRECIO TOTAL':'gmv',
                         'ID ORDEN':'orders'}, inplace=True)

rappi_status = ['Finalizado', 'pending_review', 'finished']

rappidf = rappidf[rappidf.ESTADO.isin(rappi_status)]

# eliminating unused columns
rappidf = rappidf[['created_month', 'created_week', 'facility_name', 'organization_name', 'gmv', 'orders']]


# converting to numeric type the gmv column. All errors will be set as NAN
rappidf['gmv'] = pd.to_numeric(rappidf['gmv'] ,errors='coerce')


rappidf.created_month = pd.to_datetime(rappidf.created_month, errors='coerce')
rappidf.created_week = pd.to_datetime(rappidf.created_week, errors='coerce')


rappidf = rappidf.groupby(['created_month', 'created_week',
                      'facility_name',
                      'organization_name']
                     , as_index=False
                     , dropna=False
                     ).agg({'orders': 'nunique',
                            'gmv': 'sum'}).reset_index(drop=True)


rappidf['region'] = 'LATAM'
rappidf['country'] = 'Colombia'
rappidf['tenant_type'] = 'VIRTUAL'
# rappidf['is_new_store'] = False
rappidf['all_orders'] = rappidf.orders
rappidf['cancelled_orders'] = 0
rappidf['all_gmv'] = rappidf.gmv
rappidf['cancelled_gmv'] = 0
rappidf['aov'] = rappidf.gmv / rappidf.orders
rappidf['all_gmv_usd'] = np.nan
rappidf['gmv_usd'] = np.nan
rappidf['cancelled_gmv_usd'] = np.nan
rappidf['aov_usd'] = np.nan
rappidf['facility_name'] = rappidf['facility_name'].apply(facility)
rappidf['total'] = rappidf.gmv
rappidf['organization_id'] = np.nan
rappidf['net_payout'] = rappidf.gmv
rappidf['net_gmv'] = rappidf.gmv
rappidf['discount'] = 0

# eliminating rows without date or facility name
rappidf.dropna(subset=['facility_name', 'created_month'], inplace=True)

rappidf['city'] = rappidf['facility_name'].apply(city)


# ordering the columns to match the Presto query result
rappidf = rappidf[[c for c in latam.columns]]


In [14]:
# dfs = [latam, lighttech]
# concating the 
final = pd.concat([latam, lighttech, rappidf], ignore_index=True)

In [15]:
# grouping the concatenated DataFrame
final = final.groupby(['created_month', 'created_week','region', 'country', 'city',
                      'facility_name',
                      'organization_name', 'organization_id','tenant_type']
                     , as_index=False
                     , dropna=False
                     ).sum().reset_index(drop=True)


<ipython-input-15-fad014717a25>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  final = final.groupby(['created_month', 'created_week','region', 'country', 'city',
<ipython-input-15-fad014717a25>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  final = final.groupby(['created_month', 'created_week','region', 'country', 'city',


In [16]:
# getting exchange rate
gc = pygsheets.authorize(service_file=g_sheets_creds_path)
ex = gc.open_by_key('1UNylsHToNDVNfrTC9Le5oRu0rUlA1edkW8RuSAPPtV8')
exws = ex.worksheet('title','latam_monthly')
latam_monthly = exws.get_as_df(has_header=True,start="A1", end=None,empty_value='')
latam_monthly = pd.melt(latam_monthly, id_vars='Month', var_name='country', value_name='exchange_rate')
latam_monthly.Month = latam_monthly.Month.astype('datetime64[ns]')
latam_monthly.exchange_rate = pd.to_numeric(latam_monthly.exchange_rate ,errors='coerce')
latam_monthly.rename(columns={'Month': 'created_month'}, inplace=True)

# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1rEQZXWO2GfxKWPApn7OFQ-JTOUTe_7PdAsOd8Ma2fEI')

exchange = {'Mexico':'MXN',
           'Panama':'USD',
           'Colombia':'COP',
           'Ecuador':'USD',
           'Costa Rica':'CRC',
           'Peru':'PEN',
           'Chile':'CLP',
           'Brazil':'BRL'
           }

latam_monthly['currency_code'] = latam_monthly.country.map(exchange)
latam_monthly = latam_monthly[['created_month', 'country', 'currency_code', 'exchange_rate']]

ws = ss.worksheet('title','exchange_rate')

print('Clearing Range ...')
ws.clear(start='B', end='D')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(latam_monthly, start="B1",copy_head=True, extend=True, nan='')
print('Exchange Rate UPDATED!!')

final = final.merge(latam_monthly, on=['created_month','country'], how='left')
final['all_gmv_usd'] = final.all_gmv / final.exchange_rate
final['cancelled_gmv_usd'] = final.cancelled_gmv / final.exchange_rate
final['gmv_usd'] = final.gmv / final.exchange_rate
final['aov_usd'] = final.aov / final.exchange_rate


final.drop(['exchange_rate'], axis=1, inplace=True)


Clearing Range ...
Range CLEARED
Pasting Info ...
Exchange Rate UPDATED!!


In [17]:
final = final.sort_values(by=['country','organization_name', 'organization_id'], ascending=True, na_position='last').reset_index(drop=True)

final.organization_id = final.organization_id.ffill()

final = final[final.created_month <= np.datetime64('today')]

# final.to_csv('/home/shared/esteban.cardona/latam_orders_gmv/csv/latam_orders_gmv.csv', index=False)

final = final[final.created_month >= '2020-08-01']


In [18]:
final = final[[c for c in latam.columns]]

In [19]:
final = final.sort_values(by=['created_week','country', 'city', 'facility_name'], ascending=True, na_position='last').reset_index(drop=True)

final['net_gmv'] = final.gmv - final.discount

In [20]:
centrito_nano = [{'created_month':datetime(2021, 10, 1), 
'created_week':np.nan,
'region':'LATAM',
'country':'Mexico', 
'city':'Monterrey',
'facility_name':'MTY - Centrito NK', 
'organization_name':np.nan,
'organization_id':np.nan,
'tenant_type':'DELIVERY',
'all_orders':6774,
'cancelled_orders':0,
'orders':6774, 
 'all_gmv':2147358,
'cancelled_gmv':0,
'gmv':2147358,
'aov':np.nan,
'all_gmv_usd':104997, 
'cancelled_gmv_usd':0, 
'gmv_usd':104997,
'aov_usd':np.nan,
'net_payout':np.nan,
'discount':np.nan, 
'net_gmv':np.nan},

{'created_month':datetime(2021, 11, 1), 
'created_week':np.nan,
'region':'LATAM',
'country':'Mexico', 
'city':'Monterrey',
'facility_name':'MTY - Centrito NK', 
'organization_name':np.nan,
'organization_id':np.nan,
'tenant_type':'DELIVERY',
'all_orders':3387,
'cancelled_orders':0,
'orders':3387, 
 'all_gmv':1073679,
'cancelled_gmv':0,
'gmv':1073679,
'aov':np.nan,
'all_gmv_usd':51439, 
'cancelled_gmv_usd':0, 
'gmv_usd':51439,
'aov_usd':np.nan,
'net_payout':np.nan,
'discount':np.nan, 
'net_gmv':np.nan}]


In [21]:
for month in centrito_nano:
    final = final.append(month, ignore_index=True)

<ipython-input-21-576bd99b5534>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(month, ignore_index=True)
<ipython-input-21-576bd99b5534>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(month, ignore_index=True)


In [22]:
final.created_month.value_counts()

2022-03-01    3956
2022-01-01    3891
2022-02-01    3534
2021-12-01    3516
2021-11-01    3397
2021-08-01    3328
2021-10-01    3217
2021-09-01    3034
2021-05-01    3021
2021-07-01    2864
2021-06-01    2777
2021-04-01    2521
2021-03-01    2273
2022-04-01    2243
2021-01-01    1636
2021-02-01    1619
2020-12-01    1327
2020-11-01    1318
2020-10-01     940
2020-09-01     767
2020-08-01     737
Name: created_month, dtype: int64

In [23]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(service_file=g_sheets_creds_path)


# open the spreedsheet
print('Opening Spreadsheet')



# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1XfuB4pnAt5LiZmH7zCjNochwLHFTlCqHXS2SNiKEa0U')

# select sheet

ws = ss.worksheet('title','superset_raw')

print('Clearing Range ...')
ws.clear(start='A', end='W')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final, start="A1",copy_head=True, extend=True)
print('LATAM KPIs UPDATED!!\n')


# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1PsOJjzhtK2zjgg349MWf9W4kI2gVmQ77LUTtfPQ7Ksw')

# select sheet

ws = ss.worksheet('title','superset_raw')

print('Clearing Range ...')
ws.clear(start='A', end='W')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final, start="A1",copy_head=True, extend=True)
print('LATAM ORDER VOLUME UPDATED!!')



Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
LATAM KPIs UPDATED!!

Clearing Range ...
Range CLEARED
Pasting Info ...
LATAM ORDER VOLUME UPDATED!!


In [24]:
# # Everything ready to upload to GSheets

# # open the spreedsheet
# print('Opening Spreadsheet')

# # 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
# ss = gc.open_by_key('1W6a8o8E7lLPtjgMm-2k0HLUC3XHhNH2iiOjwL_qyaWM')

# # select sheet

# ws = ss.worksheet('title','superset_raw')

# print('Clearing Range ...')
# ws.clear(start='A', end='W')
# print('Range CLEARED')

# print('Pasting Info ...')
# ws.set_dataframe(final, start="A1",copy_head=True, extend=True)
# print('CLOUD RETAIL DASH UPDATED!!')

In [25]:
# Everything ready to upload to GSheets

# open the spreedsheet
print('Opening Spreadsheet')

# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1-ZH_mqlzyOttNkwlT1jqXVwi_1CZbWGbE8h1x6ckFaI')

# select sheet

ws = ss.worksheet('title','superset_raw')

print('Clearing Range ...')
ws.clear(start='A', end='W')
print('Range CLEARED')

final = final[final.created_month >= '2021-08-01']

print('Pasting Info ...')
ws.set_dataframe(final, start="A1",copy_head=True, extend=True)
print('WEEKLY MAIL UPDATED!!')

Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
WEEKLY MAIL UPDATED!!


# WHEN I WORK

In [26]:
facilities_sfdc =  { 4574219: 'BOG - Puente Aranda'
                    ,4574220: 'BOG - Prado'
                    ,4782885: 'BOG - Castellana'
                    ,4841113: 'MEX - Polanco'
                    ,4849090: 'MEX - Insurgentes Sur'
                    ,4849091: 'MEX - Napoles'
                    ,4873511: 'MTY - Tec'
                    ,4873514: 'MTY - Santa Maria'
                    ,4873516: 'MTY - Cumbres'
                    ,4873517: 'MTY - Valle Oriente'
                    ,4887049: 'CLO - Chipichape'
                    ,4887913: 'MEX - Pedregal'
                    ,4951568: 'BOG - Cocinas de la 73'
                    ,4960004: 'BOG - Atabanza'
                    ,4966554: 'MEX - Santa Fe'
                    ,4978292: 'BOG - San Felipe'
                    ,4983273: 'MEX - Roma'
                    ,5007568: 'BAQ - Alto Prado'
                    ,5008009: 'MTY - Leones NK'
                    ,5008011: 'MTY - TEC NK'
                    ,5008012: 'MTY - Valle Oriente NK'
                    ,5015102: 'MED - Poblado 1 EM'
                    ,5015105: 'MED - Poblado 2 EM'
                    ,5015106: 'MED - Envigado 1 EM'
                    ,5015108: 'MED - Laureles EM'
                    ,5015109: 'BOG - Salitre 1 EM'
                    ,5015110: 'BOG - Salitre 2 EM'
                    ,5015111: 'BOG - 116 EM'
                    ,5015112: 'BOG - 134 EM'
                    ,5015113: 'BAQ - El Tabor EM'
                    ,5033231: 'GDL - Ciudad del Sol'
                    ,5051729: 'LIM - Surquillo'
                    ,5053715: 'SJO - Guadalupe'
                    ,5091431: 'UIO - 10 de Agosto'
                    ,5099120: 'BOG - Cedritos'
                    ,5099861: 'MEX - Lindavista'
                    ,5102666: 'SJO - Pavas'
                    ,5103181: 'SJO - Heredia'
                    ,5135127: 'LIM - Magdalena'
                    ,5139303: 'LIM - Lince'
                    ,5145149: 'SCL - Recoleta'
                    ,4970206: 'BHZ - Arthur Guimaraes'
                    ,4970210: 'BHZ - Paracatu'
                    ,4970211: 'BHZ - Contagem Industrial'
                    ,4984387: 'SAO - Lapa Clelia'
                    ,4984440: 'SAO - Rua do Acre'
                    ,5019156: 'RIO - Jose Bonifacio'
                    ,5037826: 'SAO - Dino'
                    ,5037827: 'SAO - Guararapes (Brooklin)'
                    ,5058563: 'SAO - Vargas'
                    ,5067963: 'BHZ - Tancredo Neves'
                    ,5072437: 'RIO - Barra'
                    ,5090401: 'SAO - Borba'
                    ,5091759: 'RIO - Camerino'
                    ,5146430: 'GDL - Av. Mexico'
                    ,5158983: 'RIO - Niteroi'
                    ,5164867: 'MED - DModa'
                    ,5203329: 'BSB - Aguas Claras'
                  }
        

def pais(x):
    if x.startswith(('MEX', 'MTY', 'GDL')):
        return 'Mexico'
    elif x.startswith(('BOG', 'CLO', 'BAQ', 'MED')):
        return 'Colombia'
    elif x.startswith(('SJO')):
        return 'Costa Rica'
    elif x.startswith('LIM'):
        return 'Peru'
    elif x.startswith('UIO'):
        return 'Ecuador'
    elif x.startswith('SCL'):
        return 'Chile'
    elif x.startswith(('SAO', 'RIO', 'BHZ', 'BSB')):
        return 'Brazil'
    else:
        return x
    
    
    
def city(x):
    if x.startswith('MEX'):
        return 'Mexico City'
    elif x.startswith('MTY'):
        return 'Monterrey'
    elif x.startswith('GDL'):
        return 'Guadalajara'
    elif x.startswith('BOG'):
        return 'Bogota'
    elif x.startswith('CLO'):
        return 'Cali'
    elif x.startswith('SJO'):
        return 'San Jose - CR'
    elif x.startswith('LIM'):
        return 'Lima'
    elif x.startswith('UIO'):
        return 'Quito'
    elif x.startswith('BAQ'):
        return 'Barranquilla'
    elif x.startswith('SCL'):
        return 'Santiago'
    elif x.startswith('SAO'):
        return 'Sao Paulo'
    elif x.startswith('BHZ'):
        return 'Belo Horizonte'
    elif x.startswith('RIO'):
        return 'Rio de Janeiro'
    elif x.startswith('BSB'):
        return 'Brasilia'
    elif x.startswith('MED'):
        return 'Medellin'
    else:
        return x


wiw_creds = json.load(open(wiw_creds_path))

creds = wiw_creds['creds']


In [27]:
final_latam = pd.DataFrame()
lista = []

for c in creds:
    #### AUTH
    _base_url = 'https://api.login.wheniwork.com/'
    Body = {
                'Email': str(c['email']),
                'Password': str(c['password'])
            }
    endpoint = 'login'

    #Request Data
    token = requests.post(_base_url + endpoint, data = Body)

    token_j = token.json()

    token_api = str(token_j['token'])

    print('auth ok')
    ## SHIFTS

    #### Define Request Parameters
    _base_url = 'https://api.wheniwork.com/'
    _api_key = token_api
    _headers = {'Authorization': 'Bearer ' + _api_key}
    
    #to modify now and end with desired timeframe
    endpoint = '2/shifts?start=2021-09-01T00:00:00Z&end=now+60 days&unpublished=True'

    #Request Data
    shifts = requests.get(_base_url + endpoint, headers=_headers)

    shifts_j = shifts.json()

    # Flatten data
    shifts_df = pd.json_normalize(shifts_j, record_path =['shifts'])

    shifts_df = shifts_df[['id', 'account_id', 'user_id', 'location_id', 'position_id', 'start_time', 'end_time','published', 'break_time']]



    ################### USERS ###################
    
    # to modify endpoint showing deleted users and pending to be activated
    endpoint = '2/users?show_deleted=true&show_pending=true'

    #Request Data
    users = requests.get(_base_url + endpoint, headers=_headers)

    users_j = users.json()

    users_df = pd.json_normalize(users_j,record_path =['users'])

    users_df['user_name'] = users_df['first_name'] +' ' + users_df['last_name']

    users_df.rename(columns={'id':'user_id'
                             }, inplace=True)

    users_df = users_df[['user_id','user_name', 'is_active']]



    ################### POSITIONS ###################

    endpoint = '2/positions?show_deleted=true'

    #Request Data
    positions = requests.get(_base_url + endpoint, headers=_headers)

    positions_j = positions.json()

    positions_df = pd.json_normalize(positions_j,record_path =['positions'])

    positions_df.rename(columns={'id':'position_id',
                                 'name':'position_name'
                             }, inplace=True)

    positions_df = positions_df[['position_id','position_name']]
    
    display(positions_df)
    
    

    ################### LOCATIONS ###################
    endpoint = '2/locations'

    #Request Data
    locations = requests.get(_base_url + endpoint, headers=_headers)

    locations_j = locations.json()

    locations_df = pd.json_normalize(locations_j,record_path =['locations'])

    locations_df.rename(columns={'id':'location_id',
                                 'name':'location_name'
                             }, inplace=True)

    locations_df = locations_df[['location_id','location_name']]
    

    
    #mapping facility names with sfdc nomenclature
    locations_df['facility_name'] = locations_df.location_id.map(facilities_sfdc)
    display(locations_df)
    
    
    ################### MERGING ###################
    when = shifts_df.merge(users_df, on='user_id', how='left')
    when = when.merge(positions_df, on= 'position_id', how='left')
    when = when.merge(locations_df, on= 'location_id', how='left')

    when.start_time = pd.to_datetime(when.start_time)
    when.end_time = pd.to_datetime(when.end_time)
    when['start_month'] = when.start_time.dt.to_period('M').dt.to_timestamp()
    # getting first day of week
    when['dayofweek'] = when.start_time.dt.weekday
    when['start_week'] = (when.start_time - when.dayofweek * timedelta(days=1)).dt.date
    when['shift_hours'] =  (when.end_time - when.start_time) / np.timedelta64(1, 'h')
    when['final_hours'] = when.shift_hours - when.break_time
    
    when.drop(['dayofweek'], axis=1, inplace=True)
    
    when['facility_name'] = when['facility_name'].astype(str)

    when['country'] = when.facility_name.apply(pais)
    when['city'] = when.facility_name.apply(pais)

    lista.append(when)
    
    final_latam = pd.concat([final_latam,when], ignore_index=True)
    
    
    
    
final_latam = final_latam[final_latam.start_month >= '2021-09-01']

include = [9236625, 10115677, 10145881, 10283712, 10075194, 10075195, 0]

runner = [9236625, 10075194]

final_latam = final_latam[final_latam.position_id.isin(include)]

final_latam['runner'] = final_latam.position_id.isin(runner)

unpublished = final_latam[~final_latam.published].reset_index(drop=True)

no_position = final_latam[final_latam.position_name.isna()].reset_index(drop=True)

final_latam = final_latam.dropna(subset=['position_name'])

final_latam = final_latam[final_latam.published]

final_agg = final_latam.groupby(['start_month', 'start_week','country', 'city', 'facility_name','runner'], as_index=False
                                                                                                     , dropna=False
                                                                                                     ).agg({'final_hours': 'sum'}).reset_index(drop=True)

final_latam = final_latam[['country', 'city', 'user_name', 'is_active','location_name', 'position_name',  'start_time', 'end_time', 
                           'shift_hours', 'break_time', 'final_hours', 'start_month', 'start_week', 'facility_name', 'published', 'runner']]

cols = [c for c in final_latam.columns]


# Everything ready to upload to GSheets

gc = pygsheets.authorize(service_file=g_sheets_creds_path)

# open the spreedsheet
print('Opening Spreadsheet')

# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1lxP_lIX57nLMaVbGMjl3i-s9GvUWcYOdd2k69CHfPsY')

# select sheet

ws = ss.worksheet('title','no_position_data')

print('Clearing Range ...')
ws.clear(start='A', end='V')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(no_position[cols], start="A1",copy_head=True, extend=True, nan= '' )
print('no_position UPDATED!!\n')





# select sheet

ws = ss.worksheet('title','unpublished_data')

print('Clearing Range ...')
ws.clear(start='A', end='O')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(unpublished[cols], start="A1",copy_head=True, extend=True, nan= '' )
print('unpublished_data UPDATED!!\n')






# select sheet

ws = ss.worksheet('title','RawData')

print('Clearing Range ...')
ws.clear(start='A', end='O')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final_latam, start="A1",copy_head=True, extend=True, nan= '' )
print('RawData UPDATED!!\n')






# select sheet

ws = ss.worksheet('title','RawAggregated')

print('Clearing Range ...')
ws.clear(start='A', end='G')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final_agg, start="A1",copy_head=True, extend=True, nan= '' )
print('RawAgg UPDATED!!\n')





# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1XfuB4pnAt5LiZmH7zCjNochwLHFTlCqHXS2SNiKEa0U')

# select sheet

ws = ss.worksheet('title','when_i_work_raw')

print('Clearing Range ...')
ws.clear(start='A', end='G')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final_agg, start="A1",copy_head=True, extend=True, nan= '' )
print('LATAM KPIs UPDATED!!\n')


# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1-ZH_mqlzyOttNkwlT1jqXVwi_1CZbWGbE8h1x6ckFaI')

# select sheet

ws = ss.worksheet('title','wiw_raw')

print('Clearing Range ...')
ws.clear(start='A', end='G')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final_agg, start="A1",copy_head=True, extend=True, nan= '' )
print('Weekly Mail UPDATED!!\n')




auth ok


,position_id,position_name
0,10075194,Runner
1,10075195,Shift Lead
2,10075196,Manager
3,10278403,all
4,10341681,training
5,10524142,Cloud Retail


,location_id,location_name,facility_name
0,4970206,Artur Guimarães,BHZ - Arthur Guimaraes
1,4970210,Paracatu,BHZ - Paracatu
2,4970211,David Sarnoff,BHZ - Contagem Industrial
3,4984387,Clelia - Lapa,SAO - Lapa Clelia
4,4984440,Acre - Mooca,SAO - Rua do Acre
5,5019156,Méier,RIO - Jose Bonifacio
6,5037826,Dino Bueno,SAO - Dino
7,5037827,Guararapes,SAO - Guararapes (Brooklin)
8,5058563,Vargas,SAO - Vargas
9,5067963,Tancredo Neves,BHZ - Tancredo Neves


/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pypi__pandas/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


auth ok


,position_id,position_name
0,9236625,Runner
1,9236626,Shift Lead
2,9236627,Supervisor
3,10115677,Runner cleaning
4,10140113,Vacaciones
5,10140114,Incapacidad
6,10145881,Facility Lead
7,10147498,Other
8,10258324,Conserje
9,10283712,VB Inventory


,location_id,location_name,facility_name
0,4574219,Puente Aranda,BOG - Puente Aranda
1,4574220,Prado,BOG - Prado
2,4782885,Castellana,BOG - Castellana
3,4841113,Polanco,MEX - Polanco
4,4849090,Insurgentes Sur,MEX - Insurgentes Sur
5,4849091,Napoles,MEX - Napoles
6,4873511,Tec (Pedro Martinez),MTY - Tec
7,4873514,Santa Maria (Aaron Saenz),MTY - Santa Maria
8,4873516,Cumbres (Antonio Plaza),MTY - Cumbres
9,4873517,Valle Oriente (Argentina),MTY - Valle Oriente


/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pypi__pandas/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
no_position UPDATED!!

Clearing Range ...
Range CLEARED
Pasting Info ...
unpublished_data UPDATED!!

Clearing Range ...
Range CLEARED
Pasting Info ...
RawData UPDATED!!

Clearing Range ...
Range CLEARED
Pasting Info ...
RawAgg UPDATED!!

Clearing Range ...
Range CLEARED
Pasting Info ...
LATAM KPIs UPDATED!!

Clearing Range ...
Range CLEARED
Pasting Info ...
Weekly Mail UPDATED!!



In [28]:
today = date.today()

today = today.strftime("%B %d, %Y")
# assign data of lists.  
data = {'Last_Update': [today]}  
  
# Create DataFrame  
last_update = pd.DataFrame(data)  

# last_update.to_csv('/home/shared/esteban.cardona/latam_orders_gmv/csv/last_update.csv', index=False )

display(last_update)

# os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/esteban.cardona/creds/secret.json'

# gcs_s = storage.Client()
# BUCKET ='css-growth-latam'
# mypath = '/home/shared/esteban.cardona/latam_orders_gmv/csv/'

# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# bktfolder = 'latam_orders_gmv/'

# for i in onlyfiles:
#     save_location= bktfolder + i
#     file_list = mypath + i

#     print("\n","Saving at", save_location)
        
#     gcs_s.get_bucket(BUCKET).blob(save_location).upload_from_filename(
#         file_list,content_type='text/csv')
    
    
# import datetime

# datetime.datetime.now()

,Last_Update
0,"April 07, 2022"


# Hourly process

In [29]:
# query = """
# SELECT 
# DOW(from_unixtime(reference_time/1000, facility_timezone)) as day_num,
# CASE 
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 1 THEN 'Monday'
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 2 THEN 'Tuesday'
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 3 THEN 'Wednesday'
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 4 THEN 'Thursday'
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 5 THEN 'Friday'
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 6 THEN 'Saturday'
# WHEN DOW(from_unixtime(reference_time/1000, facility_timezone)) = 7 THEN 'Sunday'
# END as day,
# HOUR(date_trunc('hour', from_unixtime(reference_time/1000, facility_timezone))) as hour,
# dc.country,
# (
# CASE
#    WHEN
#       facility_id IN 
#       (
#          'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
#          '4e36823e-98a2-4382-8797-48ef313196a9',  -- LIM - Lince
#          '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
#       )
#    THEN
#       'Lima'
#    WHEN
#       facility_id IN 
#       (
#          'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
#       )
#    THEN
#       'Quito'
#    WHEN
#       facility_id IN 
#       (
#          '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
#          '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
#          '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
#          'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
#          'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
#          '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
#          'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
#          '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82'  -- BOG - Cedritos
#       )
#    THEN
#       'Bogota'
#    WHEN
#       facility_id in 
#       (
#          '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
#       )
#    THEN
#       'Cali' 
#    WHEN
#       facility_id in 
#       (
#          '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
#       )
#    THEN
#       'Barranquilla' 
#    WHEN 
#      facility_id in
#      (
#         '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' -- MED - DModa
#      )
#    THEN
#      'Medellin'
#    WHEN
#       facility_id in 
#       (
#          '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
#          '967f0a2e-9c8e-4b35-929b-37420d9439c9'  -- SJO - Pavas
#       )
#    THEN
#       'San Jose - CR' 
#    WHEN
#       facility_id in 
#       (
#          '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
#          '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
#          '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
#          'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
#          '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
#          'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
#          'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
#       )
#    then
#       'Mexico City'
#    when
#       facility_id in 
#       (
#          '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
#          '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
#          '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
#          '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
#          '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
#       )
#    then
#       'Monterrey'
#    when
#       facility_id in 
#       (
#          'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
#          'c66ce711-5486-433f-a9b7-1b36ae3a6dee'   -- GDL - Av. Mexico
#       )
#    then
#       'Guadalajara'
#    WHEN
#       facility_id IN
#       (
#          '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
#          '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
#          '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
#          '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
#       )
#    then
#       'Belo Horizonte' 
#    WHEN
#       facility_id IN
#       (
#          '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
#          '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
#          '0ee5dccf-ed5b-465c-9858-b53351353e3b'  -- RIO - Camerino
#       )
#    then
#       'Rio de Janeiro' 
#    WHEN
#       facility_id IN
#       (
#          '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
#          '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
#          '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
#          'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
#          '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
#          '88aaaf99-a444-4e0f-99f6-267b635faa53'  -- SAO - Vargas
#           )
#    then
#       'Sao Paulo'
#     WHEN
#         facility_id IN 
#         (
#         'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
#         )
#     THEN
#         'Brasilia'
#     WHEN 
#         facility_id IN 
#         (
#           'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- SCL - Recoleta
#         )
#     THEN
#         'Santiago'
# END
# ) AS city,
# (
# CASE
#    WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
#    WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
#    WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
#    WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
#    WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
#    WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
#    WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
#    WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
#    WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
#    WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
#    WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
#    WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
#    WHEN facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
#    WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
#    WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
#    WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
#    WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
#    WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
#    WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
#    WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
#    WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
#    WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
#    WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
#    WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
#    WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
#    WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
#    WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
#    WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
#    WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
#    WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
#    WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
#    WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
#    WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
#    WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
#    WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
#    WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
#    WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
#    WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
#    WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
#    WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
#    WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
#    WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
#    WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
#    WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
#    WHEN facility_id = '4e36823e-98a2-4382-8797-48ef313196a9' THEN 'LIM - Lince'
#    WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
# END
# ) AS facility_name,

# COUNT (DISTINCT order_id) AS orders

# FROM hudi_ingest.analytics_views.customer_orders co
# LEFT JOIN hudi_ingest.scratch.dim_countries dc 
#    ON co.facility_country_code = dc.country_code AND region = 'LATAM'
# WHERE day_partition >= cast(date_add('month', -3, date_trunc('month',current_timestamp)) as VARCHAR )
# AND reference_time/1000 IS NOT NULL 
# AND NOT is_cancelled
# AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= date_add('month', -2, date_trunc('month',current_timestamp))
# AND facility_id IN ( -- 
# -- ANDEAN
#     --PERU
#         --LIMA
#             'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
#             '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
#             '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
#     --ECUADOR
#         -- QUITO
#             'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
# -- CCAC
#     -- COLOMBIA
#         -- BOGOTA
#             '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
#           --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
#             '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
#             'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
#             'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
#             '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
#             'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
#             '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
#         -- CALI    
#             '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
#         -- BARRANQUILLA
#             '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
#         -- MEDELLIN
#             '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
#     -- COSTA RICA
#         -- SAN JOSE
#             '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
#             '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas

# -- MEXICO
#         -- MEXICO CITY
#             '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
#             '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
#             '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
#             'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
#             '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
#             'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe   
#         -- MONTERREY
#             '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
#             '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
#             '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
#             '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
#             '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
#         -- GUADALAJARA
#             'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
#             'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
# -- CHILE
#         -- SANTIAGO
#             'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
# -- BRAZIL
#         -- BELO HORIZONTE
#             '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
#             '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
#             '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
#         -- RIO DE JANEIRO
#             '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
#             '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
#         -- SAO PAULO
#             '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
#             '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
#             '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
#             'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
#             '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
#             '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
#         -- BRASILIA
#             'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
#                 )
# GROUP BY 1,2,3,4,5,6
# """
# print('querying ...')
# hours = ds_presto.fetch_data(conn=conn, sql_str=query, use_cache=False)
# print('pulled')


# # concatenating lighttech DataFrames
# lighttech = pd.concat(dataframes)


# # renaming columns to match Presto Query (latam df) headers
# lighttech.rename(columns={'Número \ndo pedido':'orders', 
#                          'Facility':'facility_name',
#                          'Hora de Registro':'created_date'}, inplace=True)


# # eliminating unused columns
# lighttech = lighttech[['created_date', 'orders', 'facility_name']]

# # replacing whitespaces with NaN
# lighttech = lighttech.replace(r'^\s*$', np.nan, regex=True)

# # cleaning dates
# lighttech['created_date'] = lighttech['created_date'].str.replace('/', '-').str.replace('--', '-')

# # converting to datetime
# lighttech['created_date'] = pd.to_datetime(lighttech['created_date'], errors = 'coerce')

# # eliminating rows without date or facility name
# lighttech.dropna(subset=['created_date', 'facility_name'], inplace=True)

# # getting first day of week
# lighttech['day_num'] = lighttech.created_date.dt.weekday + 1

# lighttech['day'] = lighttech.created_date.dt.day_name()

# lighttech['hour'] = lighttech.created_date.dt.hour

# # grouping to get amount of orders and gmv
# lighttech = lighttech.groupby(['facility_name', 'day_num',
#                       'day',
#                       'hour']
#                      , as_index=False
#                      , dropna=False
#                      ).agg({'orders': 'count'}).reset_index(drop=True)

# lighttech.orders = pd.to_numeric(lighttech.orders)

# # creating new columns to match the Presto query headers
# lighttech['country'] = 'Brazil'
# lighttech['facility_name'] = lighttech['facility_name'].apply(facility)
# lighttech['city'] = lighttech['facility_name'].apply(city)

# lighttech = lighttech[[c for c in hours.columns]]


# hours = pd.concat([hours, lighttech], ignore_index=True)


# hours = hours.groupby(['country', 'city',
#                       'facility_name','day_num', 'day','hour']
#                      , as_index=False
#                      , dropna=False
#                      ).sum().reset_index(drop=True)


# tot_orders = hours.pivot_table(index=['facility_name'], 
# #                    columns=['sex', 'studytime'],
#                     values=['orders'], 
#                     fill_value='', aggfunc='sum').reset_index().rename(columns={'orders': 'tot_orders'})


# hours = hours.merge(tot_orders, on = ['facility_name'], how='left')


# hours['prop'] = hours.orders/hours.tot_orders

# hours.to_csv('/home/shared/esteban.cardona/latam_orders_gmv/csv/hours.csv', index=False)


